In [1]:
import lightning.pytorch as pl
import torch
import torch.nn as nn
import logging
import anndata as ad
from protoplast.scrna.anndata.torch_dataloader import DistributedAnnDataset
from protoplast.scrna.anndata.trainer import RayTrainRunner
import numpy as np
from torch.utils.data import DataLoader
import os
from scipy import sparse
from sklearn.decomposition import IncrementalPCA
from sklearn.utils import gen_batches

✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!


## Run IncrementalPCA with PROTOplast 

### Class Description

The class **`IncrementalPCAlm(pl.LightningModule)`** replicates the [**`IncrementalPCA`** algorithm from *scikit-learn*](https://github.com/scikit-learn/scikit-learn/blob/b24c328a30/sklearn/decomposition/_incremental_pca.py#L21). However, instead of running entirely in **RAM**, it processes data in **batches on the GPU**. In addition, it supports **checkpoint saving and restoration** during **PyTorch Lightning** training

This implementation is based on the **`perturb_mean`** module from the [**ArcInstitute/state** repository](https://github.com/ArcInstitute/state/blob/b6d26731e41d78c8c789d6973fe3d7db7853e9ad/src/state/tx/models/perturb_mean.py)

**Here is the summary:**
- Each batch updates the running mean, variance, and principal components using SVD.
- After training, the model can project new data into a lower-dimensional PCA space.
- All learned PCA parameters (like components and mean) are saved in a checkpoint for later use.
- A dummy optimizer is included only to make PyTorch Lightning’s training loop run correctly.

In [2]:
# define the lightningmodule
logger = logging.getLogger(__name__)
class IncrementalPCAlm(pl.LightningModule):
    def __init__(self,
                 n_components=None,        # Optional[int] | None
                 whiten=False,             # bool
                 copy=True,                # bool
                 batch_size=None):         # Optional[int] | None
        """
        Parameters from original class
        ----------
        n_components : int or None, default=None  
            Number of principal components to keep.  
            If None, it will be set to `min(n_samples, n_features)` 
            during the first `partial_fit` call.

        whiten : bool, default=False  
            When True (False by default) the ``components_`` vectors are divided
            by ``n_samples`` times ``components_`` to ensure uncorrelated outputs
            with unit component-wise variances.

        copy : bool, default=True  
            If False, input data `X` may be overwritten to save memory.  
            This is not always safe for general use.

        batch_size : int or None, default=None  
            Size of the minibatches for incremental fitting.  
            If None, the value is inferred as `5 * n_features`, balancing 
            approximation accuracy and memory consumption.
        """
        super().__init__()
        self.save_hyperparameters()

        self.n_components = n_components
        self.whiten = whiten
        self.copy = copy
        self.batch_size = batch_size
        self.dummy_param = nn.Parameter(torch.zeros(1))
    
    def partial_fit(self, X):
        """
        Incremental update for PCA from one batch X.
        Assumptions:
        - X is a dense float tensor of shape (n_samples, n_features).
        - n_features must remain the same across calls.
        Returns
        self : IncrementalPCAlm
        """
    
        # copy from sklearn
        n_samples, n_features = X.shape
        first_pass = not hasattr(self, "components_")


        if first_pass:
            self.components_ = None


        if self.n_components is None:
            if self.components_ is None:
                self.n_components_ = min(n_samples, n_features)
            else:
                self.n_components_ = self.components_.shape[0]
        elif not self.n_components <= n_features:
            raise ValueError(
                "n_components=%r invalid for n_features=%d, need "
                "more rows than columns for IncrementalPCA "
                "processing" % (self.n_components, n_features)
            )
        elif self.n_components > n_samples and first_pass:
            raise ValueError(
                f"n_components={self.n_components} must be less or equal to "
                f"the batch number of samples {n_samples} for the first "
                "partial_fit call."
            )
        else:
            self.n_components_ = self.n_components


        if (self.components_ is not None) and (
            self.components_.shape[0] != self.n_components_
        ):
            raise ValueError(
                "Number of input features has changed from %i "
                "to %i between calls to partial_fit! Try "
                "setting n_components to a fixed value."
                % (self.components_.shape[0], self.n_components_)
            )

        # This is the first partial_fit
        if not hasattr(self, "n_samples_seen_"):
            self.n_samples_seen_ = 0
            # sklearn initializes mean_/var_ to 0.0 then updates; we initialize tensors directly
            self.mean_ = torch.zeros(n_features, dtype=X.dtype, device=X.device)
            self.var_  = torch.zeros(n_features, dtype=X.dtype, device=X.device)

        # This code preserves the original incremental mean and variance update formulas from scikit-learn, 
        ## differing only by explicitly computing the current batch’s mean and variance instead of using _incremental_mean_and_var.
        batch_mean = X.mean(dim=0)
        batch_var  = X.var(dim=0, unbiased=False)

        if self.n_samples_seen_ == 0:
            col_mean = batch_mean
            col_var = batch_var
            n_total_samples = n_samples
        else:
            last_sample_count = self.n_samples_seen_
            n_total_samples = last_sample_count + n_samples
            delta = batch_mean - self.mean_
            col_mean = self.mean_ + delta * (n_samples / n_total_samples)
            col_var = (
                (self.var_ * last_sample_count)
                + (batch_var * n_samples)
                + delta.pow(2) * (last_sample_count * n_samples / n_total_samples)
            ) / n_total_samples

        # it preserves the same logic as scikit-learn but uses PyTorch tensors and GPU-safe operations
        if self.n_samples_seen_ == 0:
            Xc = X - col_mean
        else:
            col_batch_mean = batch_mean
            Xc = X - col_batch_mean
            scale_val = (self.n_samples_seen_ / n_total_samples) * n_samples
            scale = torch.sqrt(torch.tensor(scale_val, dtype=X.dtype, device=X.device))
            mean_correction = scale * (self.mean_ - col_batch_mean)


            top = self.singular_values_.unsqueeze(1) * self.components_
            Xc = torch.vstack((top, Xc, mean_correction.unsqueeze(0)))

        # This replaces SciPy’s SVD with PyTorch’s GPU-based version (Vh is equivalent to Vt in scikit-learn)
        U, S, Vh = torch.linalg.svd(Xc, full_matrices=False)

        # from sklearn: explained variance and ratio
        S2 = S.pow(2)
        explained_variance = S2 / (n_total_samples - 1)
        explained_variance_ratio = S2 / (col_var * n_total_samples).sum()

        # from sklearn: commit updated state (use exact attribute names)
        self.components_ = Vh[: self.n_components_, :]           # Vh in torch plays Vt in sklearn
        self.singular_values_ = S[: self.n_components_]
        self.mean_ = col_mean
        self.var_  = col_var
        self.explained_variance_ = explained_variance[: self.n_components_]
        self.explained_variance_ratio_ = explained_variance_ratio[: self.n_components_]

        if (self.n_components_ not in (n_samples, n_features)
                and explained_variance.numel() > self.n_components_):
            self.noise_variance_ = float(explained_variance[self.n_components_:].mean().item())
        else:
            self.noise_variance_ = 0.0

        self.n_samples_seen_ = int(n_total_samples)
        return self
        
    def transform(self, X: torch.Tensor) -> torch.Tensor:
        """
        Project X onto the principal components learned by partial_fit.

        Mirrors sklearn.IncrementalPCA.transform (dense-only here):
        X_new = (X - mean_) @ components_.T
        if whiten: X_new /= sqrt(explained_variance_)

        Parameters
        ----------
        X : torch.Tensor of shape (n_samples, n_features)

        Returns
        -------
        X_new : torch.Tensor of shape (n_samples, n_components_)
        """

        # Center input by the learned per-feature mean
        X_centered = X - self.mean_  # (n_samples, n_features)

        # Linear projection onto right singular vectors (principal axes)
        X_new = X_centered @ self.components_.T  # (n_samples, n_components_)

        # from basePCA: Whitening: divide each PC score by sqrt(explained_variance_)
        # Note: This matches sklearn, which uses sqrt of explained_variance_ (not singular_values_)
        if getattr(self, "whiten", False):
            X_new = X_new / torch.sqrt(self.explained_variance_)

        return X_new
        
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        Lightning forward pass: alias to transform for convenience.
        """
        return self.transform(X)
        
    def training_step(self, batch, batch_idx):
        X = batch if torch.is_tensor(batch) else batch["X"]
        with torch.no_grad():
            self.partial_fit(X)
        # return empty loss
        return self.dummy_param.sum() * 0
    
    def configure_optimizers(self):
        return torch.optim.SGD([self.dummy_param], lr=0.0)

    def on_save_checkpoint(self, checkpoint):
        """
        Save all Incremental PCA state into the Lightning checkpoint.
        Mirrors the style of PerturbMean's save logic.
        """
        super().on_save_checkpoint(checkpoint)

        # Convert tensors to CPU NumPy arrays for serialization
        tensor_keys = [
            "components_",
            "singular_values_",
            "mean_",
            "var_",
            "explained_variance_",
            "explained_variance_ratio_",
        ]
        for k in tensor_keys:
            if hasattr(self, k):
                v = getattr(self, k)
                checkpoint[k] = v.detach().cpu().numpy()

        # Scalars / counters
        checkpoint["noise_variance_"] = float(getattr(self, "noise_variance_", 0.0))
        checkpoint["n_samples_seen_"] = int(getattr(self, "n_samples_seen_", 0))

        # Ensure n_components_ is stored (fall back to ctor arg if first batch not seen)
        n_comp = getattr(self, "n_components_", None)
        if n_comp is None:
            n_comp = self.n_components if self.n_components is not None else 0
        checkpoint["n_components_"] = int(n_comp)

        logger.info(
            "IncrementalPCA: Saved components/mean/var/EV/EVR (+noise_var, n_seen, n_components_) to checkpoint."
        )

    def on_load_checkpoint(self, checkpoint):
        """
        Restore all Incremental PCA state from the Lightning checkpoint.
        Mirrors the style of PerturbMean's load logic.
        """
        super().on_load_checkpoint(checkpoint)

        device = getattr(self, "device", torch.device("cpu"))
        to_tensor = lambda x: torch.tensor(x, dtype=torch.float32, device=device)

        # Load tensors if present; warn if a critical one is missing
        if "components_" in checkpoint:
            self.components_ = to_tensor(checkpoint["components_"])
            logger.info(f"IncrementalPCA: Loaded components_ with shape {tuple(self.components_.shape)}")
        else:
            logger.warning("IncrementalPCA: components_ not found in checkpoint.")

        if "singular_values_" in checkpoint:
            self.singular_values_ = to_tensor(checkpoint["singular_values_"])
        if "mean_" in checkpoint:
            self.mean_ = to_tensor(checkpoint["mean_"])
        else:
            logger.warning("IncrementalPCA: mean_ not found in checkpoint. Transform will fail until fitted.")
        if "var_" in checkpoint:
            self.var_ = to_tensor(checkpoint["var_"])
        if "explained_variance_" in checkpoint:
            self.explained_variance_ = to_tensor(checkpoint["explained_variance_"])
        if "explained_variance_ratio_" in checkpoint:
            self.explained_variance_ratio_ = to_tensor(checkpoint["explained_variance_ratio_"])

        # Load scalars / counters
        self.noise_variance_ = float(checkpoint.get("noise_variance_", 0.0))
        self.n_samples_seen_ = int(checkpoint.get("n_samples_seen_", 0))

        # Keep n_components_ consistent for transform/inverse_transform
        self.n_components_ = int(checkpoint.get("n_components_", self.n_components or 0))

        logger.info(
            f"IncrementalPCA: Loaded n_components_={self.n_components_}, "
            f"n_samples_seen_={self.n_samples_seen_}, noise_variance_={self.noise_variance_}"
        )

In [3]:
def pca_metadata_cb(adata: ad.AnnData, metadata: dict) -> None:
    metadata["n_features"] = int(adata.var.shape[0])
    metadata["n_components"] = 100 # number of components need to be smaller or equal than batch size 

In [4]:
class DistributedPCAAnnDataset(DistributedAnnDataset):
    """
    Dataset for Incremental PCA (unsupervised).
    Returns {'X': <tensor>} so IncrementalPCAlm.on_fit_start can consume it.
    """

    def transform(self, start: int, end: int):
        X = super().transform(start, end)  # shape: (end - start, n_features)
        return {"X": X}

In [5]:
file_paths = ["/mnt/hdd2/tan/tahoe100m/plate3_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad"]
batch_size = 2000
test_size = 0.0
val_size = 0.0

In [6]:
from protoplast.scrna.anndata.trainer import RayTrainRunner
pcarunner_trainer = RayTrainRunner(
    Model=IncrementalPCAlm,
    Ds=DistributedPCAAnnDataset,
    model_keys=["n_components", "whiten", "copy", "batch_size"],  
    metadata_cb=pca_metadata_cb,
)

2025-10-17 13:42:10,401	INFO worker.py:1951 -- Started a local Ray instance.
2025-10-17 13:42:10,621	INFO packaging.py:588 -- Creating a file package for local module '/mnt/hdd1/dung/protoplast-ml-example'.
2025-10-17 13:42:10,755	WARNING packaging.py:430 -- File /mnt/hdd1/dung/protoplast-ml-example/.git/modules/submodules/SIMS/objects/pack/pack-682433dc4cf8becc2b44606f464dde9068565261.pack is very large (34.70MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/mnt/hdd1/dung/protoplast-ml-example/.git/modules/submodules/SIMS/objects/pack/pack-682433dc4cf8becc2b44606f464dde9068565261.pack']})`
2025-10-17 13:42:10,980	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_a51f88eaae402e50.zip' (70.21MiB) to Ray cluster...
2025-10-17 13:42:11,462	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_a51f88eaae402e50.zip'.
(raylet) warning: `VIRTUAL_ENV=/mnt/hdd1/dung/protoplast-ml-example/.venv`

(TrainTrainable pid=203115) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
(TrainTrainable pid=203115) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!


(raylet) warning: `VIRTUAL_ENV=/mnt/hdd1/dung/protoplast-ml-example/.venv` does not match the project environment path `.venv` and will be ignored; use `--active` to target the active environment instead
(RayTrainWorker pid=204267) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=203115) Started distributed worker processes: 
(TorchTrainer pid=203115) - (node_id=61d2b335212354a498285f97779804d9a5a0ce94a3a3fa496a5dc606, ip=192.168.1.226, pid=204267) world_rank=0, local_rank=0, node_rank=0


(RayTrainWorker pid=204267) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
(RayTrainWorker pid=204267) ✓ Applied AnnDataFileManager patch, AnnData cannot be imported after the patch!
(RayTrainWorker pid=204267) =========Starting the training on 0 with num threads: 4=========


(RayTrainWorker pid=204267) 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
(RayTrainWorker pid=204267) GPU available: True (cuda), used: True
(RayTrainWorker pid=204267) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=204267) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=204267) /tmp/ray/session_2025-10-17_13-42-06_620239_195249/runtime_resources/working_dir_files/_ray_pkg_a51f88eaae402e50/.venv/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3 /mnt/hdd1/dung/protoplast-ml-example/.venv/lib/pyth ...
(RayTrainWorker pid=204267) /tmp/ray/session_2025-10-17_13-42-06_620239_195249/runtime_resources/workin

...

(RayTrainWorker pid=204267) /tmp/ray/session_2025-10-17_13-42-06_620239_195249/runtime_resources/working_dir_files/_ray_pkg_a51f88eaae402e50/submodules/protoplast/src/protoplast/scrna/anndata/torch_dataloader.py:130: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
(RayTrainWorker pid=204267)   return torch.sparse_csr_tensor(
(RayTrainWorker pid=204267) /tmp/ray/session_2025-10-17_13-42-06_620239_195249/runtime_resources/working_dir_files/_ray_pkg_a51f88eaae402e50/.venv/lib/python3.11/site-packages/torch/multiprocessing/reductions.py:473: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/sr

Epoch 0: 100%|██████████| 2304/2304 [28:46<00:00,  1.33it/s, v_num=0]


(RayTrainWorker pid=204267) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/dtran/training_results/TorchTrainer_2025-10-17_13-42-24/TorchTrainer_1cf86_00000_0_2025-10-17_13-42-24/checkpoint_000000)


Epoch 0: 100%|██████████| 2304/2304 [28:47<00:00,  1.33it/s, v_num=0]


(RayTrainWorker pid=204267) `Trainer.fit` stopped: `max_epochs=1` reached.
(RayTrainWorker pid=204267) [rank0]:[W1017 14:11:49.631199332 ProcessGroupNCCL.cpp:1538] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


In [7]:
result = pcarunner_trainer.train(file_paths,
                                 batch_size,  # 2000
                                 test_size,  # 0.0
                                 val_size, # 0.0
                                 result_storage_path = "~/training_results",
                                )

Setting thread_per_worker to half of the available CPUs capped at 4
Using 1 workers with {'CPU': 4} each
=========Length of val_split 0 length of test_split 0 length of train_split 147
=========Length of after dropping remainder val_split 0 length of test_split 0 length of train_split 144
Data splitting time: 9.46 seconds
Spawning Ray worker and initiating distributed training


2025-10-17 13:42:24,472	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2025-10-17 13:42:24 (running for 00:00:00.14)
Using FIFO scheduling algorithm.
Logical resource usage: 0/96 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-10-17_13-42-06_620239_195249/artifacts/2025-10-17_13-42-24/TorchTrainer_2025-10-17_13-42-24/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-10-17 13:42:29 (running for 00:00:05.17)
Using FIFO scheduling algorithm.
Logical resource usage: 0/96 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-10-17_13-42-06_620239_195249/artifacts/2025-10-17_13-42-24/TorchTrainer_2025-10-17_13-42-24/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-10-17 13:42:34 (running for 00:00:10.23)
Using FIFO scheduling algorithm.
Logical resource usage: 5.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-10-17_13-42-06_620239_195249/artifacts/2025-10-17_13-42-24

2025-10-17 14:11:47,681	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/dtran/training_results/TorchTrainer_2025-10-17_13-42-24' in 0.0101s.
2025-10-17 14:11:47,686	INFO tune.py:1041 -- Total run time: 1763.21 seconds (1763.05 seconds for the tuning loop).


== Status ==
Current time: 2025-10-17 14:11:47 (running for 00:29:23.07)
Using FIFO scheduling algorithm.
Logical resource usage: 5.0/96 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:G)
Result logdir: /tmp/ray/session_2025-10-17_13-42-06_620239_195249/artifacts/2025-10-17_13-42-24/TorchTrainer_2025-10-17_13-42-24/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)




## Load checkpoint to transfrom data

In [8]:
ckpt_path = os.path.join(result.checkpoint.path, "checkpoint.ckpt")
ckpt_path

'/home/dtran/training_results/TorchTrainer_2025-10-17_13-42-24/TorchTrainer_1cf86_00000_0_2025-10-17_13-42-24/checkpoint_000000/checkpoint.ckpt'

In [9]:
pca = IncrementalPCAlm.load_from_checkpoint(ckpt_path)
pca.eval()

IncrementalPCAlm()

**batch size: 2000 and step is 2304 --> 2000 x 2304 = 4608000**

In [10]:
print("n_components_:", pca.n_components_)
print("n_samples_seen_:", pca.n_samples_seen_)
print("components shape:", pca.components_.shape)
print("explained_variance_ratio sum:", pca.explained_variance_ratio_.sum().item())

n_components_: 100
n_samples_seen_: 4608000
components shape: torch.Size([100, 62710])
explained_variance_ratio sum: 0.45490333437919617


In [11]:
# Move all internal PCA tensors to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for attr in ["mean_", "components_", "explained_variance_", "explained_variance_ratio_",
              "singular_values_", "var_"]:
    if hasattr(pca, attr):
        tensor = getattr(pca, attr)
        if isinstance(tensor, torch.Tensor):
            setattr(pca, attr, tensor.to(device))

In [12]:
# load plate 3
adata = ad.read_h5ad(
    '/mnt/hdd2/tan/tahoe100m/plate3_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad',
    backed="r"
)

n_obs = adata.n_obs
n_obs

4705402

In [13]:
# Define the data range to load (0 → n_obs), set the main data key "X", and select GPU if available
# Create a DistributedAnnDataset that streams mini-batches efficiently for large-scale GPU processing
indices = [[(0, n_obs)]]
metadata = {}
sparse_key = "X"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = DistributedAnnDataset(
    file_paths=file_paths,
    indices=indices,
    metadata=metadata,
    sparse_key=sparse_key,
    mini_batch_size=batch_size,
)

In [14]:
# Create a DataLoader to iterate over the pre-batched DistributedAnnDataset
# Uses all available CPU cores for parallel data loading and prefetches batches for efficiency
loader = DataLoader(
    dataset,
    batch_size=None,                      # dataset already batches with mini_batch_size
    num_workers=os.cpu_count() or 0,      # try full CPU, reduce if you see issues
    persistent_workers=False,
    prefetch_factor=1,
)

**This code can run but the memory reaches high (~90/125 GB)**

In [15]:
# Incremental PCA: Memory-Efficient Transformation
# 1. Move the trained PCA model to the target device 
# 2. Disable gradient tracking during inference to save memory
# 3. Iterate over the dataset batch by batch:
#     - Convert sparse tensors to dense if needed
#     - Move each batch to the device asynchronously
#     - Run the PCA model to get the transformed output (Z)
#     - Write each batch directly to disk to avoid memory buildup
# 4. Finally, flush the memmap file to save all results
Z_mm = np.memmap("/mnt/hdd1/dung/tahoe100/pca_ouput/Z_light_plate3.memmap", dtype=np.float32, mode="w+", shape=(n_obs, pca.n_components_))
row = 0

with torch.inference_mode():
    for i, X in enumerate(loader):
        if X.is_sparse or getattr(X, "is_sparse_csr", False):
            X = X.to_dense()
        X = X.to(device, non_blocking=True)
        Z = pca(X)
        Z_mm[row:row + Z.size(0)] = Z.cpu().numpy()
        row += Z.size(0)

del Z_mm  # flush to disk

/mnt/hdd1/dung/protoplast-ml-example/submodules/protoplast/src/protoplast/scrna/anndata/torch_dataloader.py:130: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(
/mnt/hdd1/dung/protoplast-ml-example/.venv/lib/python3.11/site-packages/torch/multiprocessing/reductions.py:473: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_compressed_tensor(


In [16]:
k = pca.n_components_  
k

100

In [17]:
n_obs  

4705402

In [18]:
# Load the saved memmap file for reading
Z = np.memmap("/mnt/hdd1/dung/tahoe100/pca_ouput/Z_light_plate3.memmap", dtype=np.float32, mode="r", shape=(n_obs, k))
# Convert the memory-mapped array to a regular NumPy array
Z_arr = np.array(Z) 
Z_arr

array([[-7.02409134e+01, -1.47408457e+01, -2.80877614e+00, ...,
         1.96723521e+00,  6.28924131e-01,  2.24048805e+00],
       [-7.90537262e+01, -3.92183828e+00, -1.20352345e+01, ...,
         2.49565983e+00,  5.44100463e-01, -1.33440936e+00],
       [-6.44386368e+01, -2.79298649e+01, -1.24581790e+00, ...,
         1.11751628e+00, -2.67246068e-01, -1.34006178e+00],
       ...,
       [-7.17579422e+01,  8.27442551e+00,  8.13151264e+00, ...,
         6.68867886e-01,  5.99386215e-01,  4.07242298e-01],
       [-5.25310898e+01, -2.42009811e+01,  6.19311392e-01, ...,
         6.17329106e-02,  6.34191394e-01, -1.61708683e-01],
       [ 1.28461275e+01, -3.66389008e+01, -1.08682318e+01, ...,
        -8.03266764e-01,  3.92352670e-01,  5.20001054e-01]],
      shape=(4705402, 100), dtype=float32)

# Use IncrementalPCA from sklearn

**If I load the full dataset and run IncrementalPCA as in the tutorial, it will crash**

In [6]:
adata = ad.read_h5ad(
    '/mnt/hdd2/tan/tahoe100m/plate3_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad')
adata.X

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 5684054105 stored elements and shape (4705402, 62710)>

**Crashed!**

In [ ]:
n_components = 100
ipca = IncrementalPCA(n_components=n_components, batch_size=2000)
X_ipca = ipca.fit_transform(adata.X)
X_ipca 